In [ ]:
!pip install tensorflow gym keras-rl2 gym[atari,accept-rom-license]==0.21.0 pyglet

In [ ]:
import gym
import random

In [ ]:
env=gym.make('SpaceInvaders-v0',render_mode='human')
height,width,channels=env.observation_space.shape
actions=env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
episodes=5
for episode in range(episodes):
    state=env.reset()
    done=False
    score=0
    
    while not done:
        env.render()
        action=random.choice([0,1,2,3,4,5])
        n_state,reward,done,info=env.step(action)
        score+=reward
    print("Episode:{} Score:{}".format(episode,score))
env.close()

C:\Users\HMTyl\anaconda3\lib\site-packages\gym\envs\atari\environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(
C:\Users\HMTyl\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:434: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


Episode:0 Score:55.0
Episode:1 Score:320.0
Episode:2 Score:35.0
Episode:3 Score:490.0
Episode:4 Score:250.0


In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=32,kernel_size=(8,8),strides=(4,4),activation='relu',input_shape=(3,height,width,channels)),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4),strides=(2,2),activation='relu'),
    tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(actions,activation='linear')
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 3, 51, 39, 32)     6176      
                                                                 
 conv2d_4 (Conv2D)           (None, 3, 24, 18, 64)     32832     
                                                                 
 conv2d_5 (Conv2D)           (None, 3, 22, 16, 64)     36928     
                                                                 
 flatten_1 (Flatten)         (None, 67584)             0         
                                                                 
 dense_3 (Dense)             (None, 512)               34603520  
                                                                 
 dense_4 (Dense)             (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 6)                

In [ ]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy

In [ ]:
def build_agent(model,actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(),attr='eps',value_max=1.,value_min=.1,value_test=.2,nb_steps=10000)
    memory = SequentialMemory(limit=1000,window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True,dueling_type='avg',
                  nb_actions=actions,nb_steps_warmup=10000)
    return dqn

In [ ]:
dqn = build_agent(model,actions)
dqn.compile(tf.keras.optimizers.Adam(learning_rate=1e-4))
dqn.fit(env,nb_steps=10000,visualize=False,verbose=2)

C:\Users\HMTyl\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 10000 steps ...


C:\Users\HMTyl\anaconda3\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  487/10000: episode: 1, duration: 69.329s, episode steps: 487, steps per second:   7, episode reward: 135.000, mean reward:  0.277 [ 0.000, 30.000], mean action: 2.616 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 1531/10000: episode: 2, duration: 139.902s, episode steps: 1044, steps per second:   7, episode reward: 280.000, mean reward:  0.268 [ 0.000, 30.000], mean action: 2.618 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 2787/10000: episode: 3, duration: 168.993s, episode steps: 1256, steps per second:   7, episode reward: 315.000, mean reward:  0.251 [ 0.000, 30.000], mean action: 2.698 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 3813/10000: episode: 4, duration: 138.168s, episode steps: 1026, steps per second:   7, episode reward: 540.000, mean reward:  0.526 [ 0.000, 30.000], mean action: 2.906 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
 4674/10000: episode: 5, duration: 114.377s, episode steps: 861, steps per second:   8, episode reward: 150.00

In [ ]:
scores = dqn.test(env,nb_episodes=10,visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...


C:\Users\HMTyl\anaconda3\lib\site-packages\gym\envs\atari\environment.py:267: UserWarning: WARN: We strongly suggest supplying `render_mode` when constructing your environment, e.g., gym.make(ID, render_mode='human'). Using `render_mode` provides access to proper scaling, audio support, and proper framerates.
  logger.warn(


Episode 1: reward: 30.000, steps: 470
Episode 2: reward: 150.000, steps: 551
Episode 3: reward: 160.000, steps: 513
Episode 4: reward: 60.000, steps: 448
Episode 5: reward: 385.000, steps: 711
Episode 6: reward: 65.000, steps: 394
Episode 7: reward: 170.000, steps: 901
Episode 8: reward: 170.000, steps: 664
Episode 9: reward: 35.000, steps: 400
Episode 10: reward: 135.000, steps: 630
136.0
